In [1]:
import warnings
warnings.filterwarnings("ignore")
from utils import *
import pickle
OPBG = [ 'código','age at diagnosis (months)',            
            'ntbc dosis mg/kg/day',
            'ntbc levels (dbs)',
            'sca (urine)',
            'methionine (plasma)',
            'tyrosine (plasma)',
            'phenylalanine (plasma)',
            'pt (sec)',
            'bili total',
            'gpt',
            'got',
            'ggt',
            'alkaline phosphatase',
            'alfa-fetoprotein','glicemia']


# OPBG = [ 'age at diagnosis (months)',    'código',   
#                 'ntbc dosis mg/kg/day',
#                 'ntbc levels (dbs)',
#                 'sca (urine)',
#                 'methionine dbs',
#                 'tyrosine dbs',
#                 'phenylalanine dbs',
#                 'methionine (plasma)',
#                 'tyrosine (plasma)',
#                 'phenylalanine (plasma)',
#                 'inr',
#                 'pt (sec)',
#                 'bili total',
#                 'gpt',
#                 'got',
#                 'ggt',
#                 'alkaline phosphatase',
#                 'alfa-fetoprotein',]
    
df1 = pd.read_csv('data/tirosinemia.csv').loc[:,OPBG]
df2 = pd.read_csv('data/tirosinemia_italia.csv').loc[:,OPBG]
def extract_transform(path, keep_cols, cols_rows_with_missing_values, new_binary_target):
    _ , spark_df = SparkDataProcessor().process_data(
        data_csv_path= path,
        )

    return prepare_spark_df(
        spark_df=spark_df,
        keep_cols=keep_cols,
        cols_rows_with_missing_values=cols_rows_with_missing_values,
        new_binary_target = new_binary_target,
    )
compare_dataframes(df1, df2)
df1.columns, df2.columns

Both CSV files have the same number of columns: 16
Column names are the same.
Column order is the same.
All common columns have the same data types.


(Index(['código', 'age at diagnosis (months)', 'ntbc dosis mg/kg/day',
        'ntbc levels (dbs)', 'sca (urine)', 'methionine (plasma)',
        'tyrosine (plasma)', 'phenylalanine (plasma)', 'pt (sec)', 'bili total',
        'gpt', 'got', 'ggt', 'alkaline phosphatase', 'alfa-fetoprotein',
        'glicemia'],
       dtype='object'),
 Index(['código', 'age at diagnosis (months)', 'ntbc dosis mg/kg/day',
        'ntbc levels (dbs)', 'sca (urine)', 'methionine (plasma)',
        'tyrosine (plasma)', 'phenylalanine (plasma)', 'pt (sec)', 'bili total',
        'gpt', 'got', 'ggt', 'alkaline phosphatase', 'alfa-fetoprotein',
        'glicemia'],
       dtype='object'))

In [2]:

binary_target = 'Alpha-Fet'
df_train, _ = extract_transform("data/tirosinemia.csv",                 OPBG, ['alfa-fetoprotein', 'sca (urine)'],binary_target)
df_train.drop('código', axis=1, inplace=True)
df_test, df_test_raw  = extract_transform("data/tirosinemia_italia.csv", OPBG, ['alfa-fetoprotein', 'sca (urine)'],binary_target)



23/06/02 21:30:47 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 100.64.221.100 instead (on interface tailscale0)
23/06/02 21:30:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/06/02 21:30:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/06/02 21:30:49 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [3]:

Rome_mask = df_test_raw['código'].str.startswith('R')

df_test_FLOR  = encode_and_convert (df_test[~Rome_mask].drop('código', axis=1).reset_index(inplace=False, drop=True))
df_test_ROME  = encode_and_convert (df_test[Rome_mask].drop('código', axis=1).reset_index(inplace=False, drop=True))
df_test_FLOR['Alpha-Fet'] = df_test_FLOR['Alpha-Fet'].astype('int8').astype('category')
df_test_ROME['Alpha-Fet'] = df_test_ROME['Alpha-Fet'].astype('int8').astype('category')


compare_dataframes(df_test_ROME, df_test_FLOR)


Both CSV files have the same number of columns: 15
Column names are the same.
Column order is the same.
All common columns have the same data types.


In [4]:


compare_dataframes(df_train, df_test_ROME)






Both CSV files have the same number of columns: 15
Column names are the same.
Column order is the same.
All common columns have the same data types.


In [5]:
ray.shutdown()
ray_df_train = ray.put(df_train)
ray_df_test  = ray.put(df_test_ROME)

2023-06-02 21:30:52,185	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


In [6]:
features : list[str] = df_train.columns.tolist()
features.remove(binary_target)
features

['age at diagnosis (months)',
 'ntbc dosis mg/kg/day',
 'ntbc levels (dbs)',
 'sca (urine)',
 'methionine (plasma)',
 'tyrosine (plasma)',
 'phenylalanine (plasma)',
 'pt (sec)',
 'bili total',
 'gpt',
 'got',
 'ggt',
 'alkaline phosphatase',
 'glicemia']

In [7]:
%%time

studies = make_multiple_studies(
    ray_df_train, 
    features = features,
    targets = [binary_target],
    n_trials=50,
    Independent_testset = True,
    Independent_testset_df = ray_df_test
)

studies = ray.get(studies)

(make_a_study pid=1995444) [I 2023-06-02 21:30:53,808] A new study created in memory with name: ntbc levels (dbs) (Alpha-Fet)
(make_a_study pid=1995454) [I 2023-06-02 21:30:53,798] A new study created in memory with name: age at diagnosis (months) (Alpha-Fet)
(make_a_study pid=1995455) [I 2023-06-02 21:30:53,798] A new study created in memory with name: ntbc dosis mg/kg/day (Alpha-Fet)
(make_a_study pid=1995442) [I 2023-06-02 21:30:53,778] A new study created in memory with name: sca (urine) (Alpha-Fet)
(make_a_study pid=1995444) [I 2023-06-02 21:31:10,121] Trial 15 finished with values: [0.5, 0.0] and parameters: {'seed': 6009, 'max_depth': 3, 'eta': 0.05648779468458812, 'subsample': 0.21631963549408986, 'lambda': 0.14233456938590305, 'alpha': 0.8971395428400898, 'scale_pos_weight': 0.06040573017518591, 'kfold_splits': 2}. 
(make_a_study pid=1995444) [I 2023-06-02 21:31:10,137] Trial 13 finished with values: [0.7058823529411764, 0.048456478863954544] and parameters: {'seed': 7629, 'ma

CPU times: user 6.47 s, sys: 5.78 s, total: 12.3 s
Wall time: 4min 5s


In [9]:
with open('chile.pickle', 'wb') as handle:
    pickle.dump(studies, handle, protocol=pickle.HIGHEST_PROTOCOL)    